In [229]:
import pandas as pd

In [230]:
train=pd.read_csv('./train_df_errno.csv')
test=pd.read_csv('./test_df.csv')
sub=pd.read_csv('./sample_submission.csv')
age=pd.read_csv('./age_gender_info.csv')
train.shape, test.shape, sub.shape, age.shape

((2896, 15), (1008, 14), (150, 2), (16, 23))

In [231]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2896 entries, 0 to 2895
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   단지코드        2896 non-null   object 
 1   총세대수        2896 non-null   int64  
 2   임대건물구분      2896 non-null   object 
 3   지역          2896 non-null   object 
 4   공급유형        2896 non-null   object 
 5   전용면적        2896 non-null   float64
 6   전용면적별세대수    2896 non-null   int64  
 7   공가수         2896 non-null   float64
 8   자격유형        2896 non-null   object 
 9   임대보증금       2327 non-null   object 
 10  임대료         2327 non-null   object 
 11  10분내지하철수    2685 non-null   float64
 12  10분내버스정류장수  2892 non-null   float64
 13  단지내주차면수     2896 non-null   float64
 14  등록차량수       2896 non-null   float64
dtypes: float64(6), int64(2), object(7)
memory usage: 339.5+ KB


### 결측치

In [232]:
train.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         569
임대료           569
10분내지하철수      211
10분내버스정류장수      4
단지내주차면수         0
등록차량수           0
dtype: int64

In [233]:
test.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         180
임대료           180
10분내지하철수       38
10분내버스정류장수      0
단지내주차면수         0
dtype: int64

### 1.자격유형

In [234]:
test.loc[test['자격유형'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0


In [235]:
grouped=test.groupby(['단지코드','임대건물구분','지역','공급유형'])
group1=grouped.get_group(('C2411','아파트','경상남도','국민임대'))
group2=grouped.get_group(('C2253','아파트','강원도','영구임대'))
print(group1)
print(group2)

      단지코드  총세대수 임대건물구분    지역  공급유형   전용면적  전용면적별세대수   공가수 자격유형     임대보증금  \
193  C2411   962    아파트  경상남도  국민임대  39.43        56  25.0    A  11992000   
194  C2411   962    아파트  경상남도  국민임대  39.72       336  25.0    A  11992000   
195  C2411   962    아파트  경상남도  국민임대  39.82       179  25.0    A  11992000   
196  C2411   962    아파트  경상남도  국민임대  46.90       240  25.0  NaN  71950000   
197  C2411   962    아파트  경상남도  국민임대  51.93       150  25.0    A  21586000   

        임대료  10분내지하철수  10분내버스정류장수  단지내주차면수  
193  100720       0.0         2.0    840.0  
194  100720       0.0         2.0    840.0  
195  100720       0.0         2.0    840.0  
196   37470       0.0         2.0    840.0  
197  171480       0.0         2.0    840.0  
      단지코드  총세대수 임대건물구분   지역  공급유형   전용면적  전용면적별세대수  공가수 자격유형    임대보증금    임대료  \
258  C2253  1161    아파트  강원도  영구임대  26.37       745  0.0  NaN  2249000  44770   
259  C2253  1161    아파트  강원도  영구임대  31.32       239  0.0    C  3731000  83020   
260  C2253  1161    아파트 

In [236]:
test.loc[196,'자격유형']='A'
test.loc[258,'자격유형']='C'

In [237]:
test['자격유형'].isnull().sum()

0

In [238]:
train['자격유형'].unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O'], dtype=object)

In [239]:
mapping = { 'A':1, 'B':2, 'C':3, 'D':4, 'E':5,
            'F':6, 'G':7, 'H':8, 'I':9, 'J':10,
            'K':11, 'L':12, 'M':13, 'N':14, 'O':15  }
train['자격유형']=train['자격유형'].map(mapping).astype(int)
test['자격유형']=test['자격유형'].map(mapping).astype(int)

### 2.도보 10분거리 내 버스정류장 수_결측치

In [240]:
train.loc[train['10분내버스정류장수'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
2293,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,1,46000000,456000,NaN,NaN,1066.0,1214.0
2294,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,1,57000000,462000,NaN,NaN,1066.0,1214.0
2295,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,1,57000000,462000,NaN,NaN,1066.0,1214.0
2296,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,1,57000000,462000,NaN,NaN,1066.0,1214.0


In [241]:
grouped=test.groupby(['임대건물구분','지역','공급유형','자격유형'])
group1=grouped.get_group(('아파트','경상남도','공공임대(10년)',1))
group1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
810,C2175,595,아파트,경상남도,공공임대(10년),74.82,118,5.0,1,47000000,470000,0.0,3.0,685.0
811,C2175,595,아파트,경상남도,공공임대(10년),84.69,297,5.0,1,58000000,520000,0.0,3.0,685.0
812,C2175,595,아파트,경상남도,공공임대(10년),84.86,143,5.0,1,58000000,520000,0.0,3.0,685.0
813,C2175,595,아파트,경상남도,공공임대(10년),84.89,13,5.0,1,58000000,520000,0.0,3.0,685.0
814,C2175,595,아파트,경상남도,공공임대(10년),84.94,13,5.0,1,58000000,520000,0.0,3.0,685.0
815,C2175,595,아파트,경상남도,공공임대(10년),84.94,11,5.0,1,58000000,520000,0.0,3.0,685.0


In [242]:
train.loc[train['10분내버스정류장수'].isnull(),"10분내버스정류장수"]=5
train.loc[train['10분내버스정류장수'].isnull(),:]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수


### 3.도보 10분거리 내 지하철역 수(환승노선 수 반영)_결측치

In [243]:
train['10분내지하철수'].isnull().sum()

211

In [244]:
test['10분내지하철수'].isnull().sum()

38

In [245]:
import numpy as np

In [246]:
train.loc[train['10분내지하철수'].isnull(),:]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
86,C1312,518,아파트,충청남도,국민임대,39.72,60,12.0,1,17460000,122210,NaN,3.0,527.0,359.0
87,C1312,518,아파트,충청남도,국민임대,39.98,89,12.0,1,17460000,122210,NaN,3.0,527.0,359.0
88,C1312,518,아파트,충청남도,국민임대,41.55,225,12.0,1,19954000,130940,NaN,3.0,527.0,359.0
89,C1312,518,아파트,충청남도,국민임대,46.90,143,12.0,1,28687000,149660,NaN,3.0,527.0,359.0
90,C1874,619,아파트,충청남도,영구임대,26.37,294,2.0,3,3141000,69900,NaN,2.0,97.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,C1350,1401,아파트,대전광역시,공공임대(10년),59.91,13,2.0,1,37474000,519350,NaN,6.0,1636.0,2315.0
2318,C1350,1401,아파트,대전광역시,공공임대(10년),59.92,223,2.0,1,37385000,516130,NaN,6.0,1636.0,2315.0
2319,C1350,1401,아파트,대전광역시,공공임대(분납),51.99,146,2.0,1,54612000,131130,NaN,6.0,1636.0,2315.0
2320,C1350,1401,아파트,대전광역시,공공임대(분납),59.91,32,2.0,1,63585000,152680,NaN,6.0,1636.0,2315.0


In [247]:
train['10분내지하철수']=train['10분내지하철수'].fillna(train.groupby('10분내버스정류장수')['10분내지하철수'].transform(np.mean))
test['10분내지하철수']=test['10분내지하철수'].fillna(test.groupby('10분내버스정류장수')['10분내지하철수'].transform(np.mean))

In [248]:
print(train['10분내지하철수'].isnull().sum())
print(test['10분내지하철수'].isnull().sum())

0
0


### 공급유형_mapping

In [249]:
train['공급유형'].value_counts()

국민임대         1736
임대상가          562
행복주택          202
공공임대(10년)     182
영구임대          152
공공임대(50년)      31
공공임대(분납)       12
장기전세            9
공공분양            7
공공임대(5년)        3
Name: 공급유형, dtype: int64

In [250]:
mapping = { '국민임대':1, '임대상가':2, '행복주택':3, '공공임대(10년)':4, 
           '영구임대':5,'공공임대(50년)':6,
           '공공임대(분납)':7, '장기전세':8, '공공분양':9, '공공임대(5년)':10 }
train['공급유형'] =train['공급유형'].map(mapping).astype(int)
test['공급유형'] =test['공급유형'].map(mapping).astype(int)

### 지역_mapping

In [251]:
train['지역'].value_counts()

경기도        602
경상남도       369
부산광역시      328
대전광역시      266
강원도        187
충청북도       163
충청남도       158
전라남도       147
광주광역시      141
경상북도       111
전라북도       109
대구광역시      102
제주특별자치도     91
서울특별시       64
울산광역시       34
세종특별자치시     24
Name: 지역, dtype: int64

In [252]:
mapping1 = { '경기도':1, '경상남도':2, '부산광역시':3, '대전광역시':4, 
           '강원도':5,'충청북도':6,'충청남도':7, '전라남도':8, '광주광역시':9, 
            '경상북도':10, '전라북도':11,'대구광역시':12,'제주특별자치도':13,'서울특별시':14,'세종특별자치시':15,'울산광역시':16 }
train['지역'] =train['지역'].map(mapping1).astype(int)
test['지역'] =test['지역'].map(mapping1).astype(int)

### 임대건물구분 mapping

In [253]:
train['임대건물구분'].unique()

array(['아파트', '상가'], dtype=object)

In [254]:
mapping2 = { '아파트':1, '상가':2}
train['임대건물구분'] =train['임대건물구분'].map(mapping2).astype(int)
test['임대건물구분'] =test['임대건물구분'].map(mapping2).astype(int)

### 전용면적을 5의 배수로 변경>전용면적 상/하한 적용

In [255]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

In [256]:
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [257]:
test['전용면적'].unique()

array([ 35.,  45.,  50.,  30.,  55.,  25.,  75., 100.,  15.,  20.,  40.,
        60.,  80.,  70.])

### 머신러닝

In [258]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor

In [259]:
### 한글 폰트 설정
import matplotlib
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import platform
path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system()=="Darwin":
    rc('font', family='AppleGothic')
else:
    print("Unknown System")
matplotlib.rcParams['axes.unicode_minus'] = False

In [260]:
train.shape

(2896, 15)

In [261]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
0,C2515,545,1,2,1,30.0,276,17.0,1,9216000,82940,0.0,3.0,624.0,205.0
1,C2515,545,1,2,1,35.0,60,17.0,1,12672000,107130,0.0,3.0,624.0,205.0
2,C2515,545,1,2,1,35.0,20,17.0,1,12672000,107130,0.0,3.0,624.0,205.0
3,C2515,545,1,2,1,45.0,38,17.0,1,18433000,149760,0.0,3.0,624.0,205.0
4,C2515,545,1,2,1,45.0,19,17.0,1,18433000,149760,0.0,3.0,624.0,205.0


In [262]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [263]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2896 entries, 0 to 2895
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   단지코드        2896 non-null   object 
 1   총세대수        2896 non-null   int64  
 2   임대건물구분      2896 non-null   int32  
 3   지역          2896 non-null   int32  
 4   공급유형        2896 non-null   int32  
 5   전용면적        2896 non-null   float64
 6   전용면적별세대수    2896 non-null   int64  
 7   공가수         2896 non-null   float64
 8   자격유형        2896 non-null   int32  
 9   임대보증금       2327 non-null   object 
 10  임대료         2327 non-null   object 
 11  10분내지하철수    2896 non-null   float64
 12  10분내버스정류장수  2896 non-null   float64
 13  단지내주차면수     2896 non-null   float64
 14  등록차량수       2896 non-null   float64
dtypes: float64(6), int32(4), int64(2), object(3)
memory usage: 294.2+ KB


In [264]:
#sel=['label_generation_num','label_bus','단지내주차면수','공급유형','총세대수','전용면적','공가수','지역','임대건물구분','자격유형']
sel=['총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '10분내지하철수', '10분내버스정류장수', '단지내주차면수']
X=train[sel]
y=train['등록차량수']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=0)

### 정규화

In [265]:
# nor_X = MinMaxScaler().fit_transform(X)  # 입력 데이터 정규화
# print("정규화 : ", nor_X.shape, y.shape)

In [266]:
# # X_train, X_test, y_train, y_test = train_test_split(nor_X, y, test_size=0.3,
#                                                    random_state=0)

### 모델만들기

In [267]:
model = GradientBoostingRegressor(max_depth=3,random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
pred[0:10]

array([2275.30628111,  376.21819805,  713.81161547,  488.23819235,
         88.93564829,  707.67661259,  513.15884048,  261.51844987,
        552.77540859,  492.42106498])

In [268]:
print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.9549763891793135
테스트(score) : 0.9491070691390533


In [269]:
#mae_val=sum(abs(y_test-pred))/len(pred)
mae_val=np.mean(abs(y_test-pred))
mae_val

72.4392897012935

In [270]:
#mse_val=sum((y_test-pred)**2)/len(pred)
mse_val=np.sum((y_test-pred)**2)/len(pred)
mse_val

10178.303355612352

In [271]:
#rmse_val=mse_val ** 0.5
rmse_val=np.sqrt(mse_val)
rmse_val

100.88757780625102

In [272]:
importance_df=pd.DataFrame(model.feature_importances_,index=X[sel].columns)
importance_df.sort_values(by=0,ascending=False)

,0
단지내주차면수,0.781094
공가수,0.092965
총세대수,0.050615
공급유형,0.021269
지역,0.015341
자격유형,0.012088
전용면적,0.009940
10분내버스정류장수,0.008981
10분내지하철수,0.005803
전용면적별세대수,0.001905


### 제출파일 만들기

In [273]:
print(X_test.shape)
print(test.shape)

(724, 11)
(1008, 14)


In [274]:
sel=['총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '10분내지하철수', '10분내버스정류장수', '단지내주차면수']
X_test=test[sel]
pred = model.predict(X_test)

In [275]:
test['등록차량수'] = pred
test['단지별차량수평균'] = test.groupby("단지코드")['등록차량수'].transform(np.mean)
test_new = test.drop_duplicates(['단지코드'], keep='first').reset_index()
test_new.head()

,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,단지별차량수평균
0,0,C1072,754,1,1,1,35.0,116,14.0,8,22830000,189840,0.0,2.0,683.0,662.975451,691.449941
1,8,C1128,1354,1,1,1,35.0,368,9.0,8,22830000,189840,0.0,3.0,1216.0,1262.566837,1268.383299
2,17,C1456,619,1,3,1,30.0,82,18.0,1,19706000,156200,0.0,16.0,547.0,568.331450,587.339864
3,26,C1840,593,1,11,1,35.0,253,7.0,1,14418000,108130,0.0,3.0,543.0,561.142455,574.368937
4,30,C1332,1297,1,1,1,35.0,282,11.0,8,28598000,203050,0.0,2.0,1112.0,1024.791880,1033.243718


### 제출용 오류 코드를 0으로 만들고 150행만들기

In [276]:
add_dat={'code':['C2675','C2335','C1327'],'num':['0','0','0']}
add_df=pd.DataFrame(add_dat)
add_df

,code,num
0,C2675,0
1,C2335,0
2,C1327,0


In [277]:
sub_df = test_new[ ['단지코드', '단지별차량수평균']]
sub_df.columns = ['code', 'num']
sub_df = pd.concat([sub_df, add_df]).reset_index()
sub_df = sub_df.drop(['index'], axis=1)
sub_df

,code,num
0,C1072,691.449941
1,C1128,1268.383299
2,C1456,587.339864
3,C1840,574.368937
4,C1332,1033.243718
...,...,...
145,C1267,395.725366
146,C2189,166.403895
147,C2675,0
148,C2335,0


In [278]:
sub_df.to_csv('0719_gradient_boosting.csv', index=False)
sub_df.head()

,code,num
0,C1072,691.449941
1,C1128,1268.383299
2,C1456,587.339864
3,C1840,574.368937
4,C1332,1033.243718
